In [1]:
import pandas as pd
from myUtils import pickle_load, symb_perf_stats_vectorized

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [2]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''
  # %%timeit
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(df_close)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # print(f'df_perf mean:\n{df_perf[_cols].mean()}\n')
  # print(f'df_perf std:\n{df_perf[_cols].std()}')
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_std = grp_CAGRUI_mean / grp_CAGRUI_std
  grp_CAGRUI_mean_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_std = grp_CAGRStd_mean / grp_CAGRStd_std
  grp_CAGRStd_mean_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_std = grp_StdUI_mean / grp_StdUI_std
  grp_StdUI_mean_std

  return df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std

In [3]:
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_c)
print(f'grp_StdUI_mean_std :{grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std :{grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std :2.082762
grp_CAGRStd_mean_std :0.006427
grp_CAGRUI_mean_std :0.238902


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,A,2021-12-01,2022-11-30,1.0,0.051085,0.182175,0.122210,2.294546,0.280417
1,AA,2021-12-01,2022-11-30,1.0,0.153521,0.384052,0.108227,3.693527,0.399741
2,AAL,2021-12-01,2022-11-30,1.0,-0.113636,0.240267,0.149106,-3.171962,-0.472959
3,AAON,2021-12-01,2022-11-30,1.0,0.031701,0.292742,0.084356,1.283734,0.108291
4,AAP,2021-12-01,2022-11-30,1.0,-0.287721,0.197549,0.120304,-12.106427,-1.456457


In [4]:
# _cols2 = ['TSBK', 'ELF', 'AVEO', 'ASC', 'TPL', 'AXON', 'NRIM', 'AAON', 'UFPT', 'GILD', 'MOD', 'SANM', 'INSW', 'LOPE', 'AE', 'ABMD', 'RMBS', 'WNC']
_cols2 = ['AE', 'ABMD', 'RMBS', 'WNC']
df_temp = df_c[_cols2].copy()
# df_perf, grp_CAGRUI_mean_std = perf_eval(df_temp)
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_temp)
# print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: 4.459864
grp_CAGRStd_mean_std: 2.356705
grp_CAGRUI_mean_std: 2.066555


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,AE,2021-12-01,2022-11-30,1.0,0.480208,0.135772,0.181695,19.466059,3.536877
1,ABMD,2021-12-01,2022-11-30,1.0,0.235779,0.225901,0.188850,5.526778,1.043729
2,RMBS,2021-12-01,2022-11-30,1.0,0.408957,0.202205,0.129424,15.626818,2.022489
3,WNC,2021-12-01,2022-11-30,1.0,0.470929,0.242475,0.121417,15.995861,1.942175


In [5]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: nan
grp_CAGRStd_mean_std: nan
grp_CAGRUI_mean_std: nan


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,SPY,2021-12-01,2022-11-30,1.0,-0.09505,0.146746,0.103921,-6.2328,-0.647716
